In [1]:
# requirements
!pip install -q rdflib sparqlwrapper owlrl gdown pandas wikipedia-api

In [2]:
import os
import pandas as pd
import rdflib
from rdflib import Graph, URIRef, Literal, BNode, Namespace, RDF, FOAF, RDFS, XSD, term 
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse
from rdflib.plugins.sparql import prepareQuery
from dateutil import parser
from rdflib import Namespace
from requests.exceptions import ReadTimeout
import time
import gdown
import wikipediaapi
import re
import owlrl

In [3]:
# create directory for data
if not os.path.exists("./data/"):
    os.mkdir("./data/")

In [4]:
# Download data
gdown.download(id="1UILSMenvqzFMsIhV6l4ZV1opJpRQZnwC", output="./data/rym_top_5000_all_time.csv", quiet=True)
gdown.download(id="1UNJ2iDX-xoWIekSA0ZwoqNOk3eAZjqrp", output="./data/albumlist.csv", quiet=True)

'./data/albumlist.csv'

# **KRR Module 2 2022/23 Project**

The project consists in the creation of an RDF knowledge graph starting from tabular data (two CSV files loaded below).

You should cover all of the following steps.

### **`1. Create an RDFLib Graph from data contained in CSV files`**

In steps:
1) Load the CSV files in two pandas DataFrames and join them
2) Create an RDFLib `Graph` and add triples from the data of the dataframes

Useful links:

- [Pandas user guide](https://pandas.pydata.org/docs/user_guide/index.html) and [documentation](https://pandas.pydata.org/docs/reference/), e.g.:
    - [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html)
    - [Join dataframes](https://pandas.pydata.org/docs/user_guide/10min.html#join)
    - [Splitting and replacing strings](https://pandas.pydata.org/docs/user_guide/text.html#splitting-and-replacing-strings)
    - [Apply a function over an axis of a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html?highlight=apply#pandas.DataFrame.apply)

In [5]:
df1 = pd.read_csv('data/albumlist.csv', encoding='mac_roman')
df1.head()

,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [6]:
df2 = pd.read_csv('data/rym_top_5000_all_time.csv')
df2.head()

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379


In [7]:
df1.shape, df2.shape

((500, 6), (5000, 9))

In [8]:
# Continue here ...
df3_join = pd.merge(df1, df2,how='outer')
df3_join.head()


,Number,Year,Album,Artist,Genre,Subgenre,Ranking,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,1967.0,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock",24.0,The Beatles,1 June 1967,"Psychedelic Pop, Pop Rock","psychedelic, playful, melodic, male vocals, wa...",4.13,"43,576",863.0
1,2.0,1966.0,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock",20.0,The Beach Boys,16 May 1966,Baroque Pop,"Wall of Sound, warm, bittersweet, love, romant...",4.18,"36,305",727.0
2,3.0,1966.0,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock",11.0,The Beatles,5 August 1966,"Pop Rock, Psychedelic Pop","psychedelic, melodic, male vocals, drugs, ecle...",4.23,"43,178",1160.0
3,4.0,1965.0,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock",35.0,Bob Dylan,30 August 1965,"Folk Rock, Singer/Songwriter","poetic, cryptic, surreal, sarcastic, male voca...",4.17,"26,338",604.0
4,5.0,1965.0,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock,73.0,The Beatles,3 December 1965,Pop Rock,"melodic, love, male vocals, bittersweet, roman...",4.03,"33,508",721.0


Create an RDFLib `Graph` and add triples from the data of the dataframes**

For example, add triples like the following (but not only):

- `[album] dbp:artist [artist]`,
- `[album] rdfs:label [album_name]`,
- `[album] rdf:type [genre]`

Make sure you use URIs and Literals correctly.

In [9]:
# Create the RDFLib Graph
rdf_graph = rdflib.Graph()

# Define your custom namespace
ex = Namespace('http://example.org/')
dbp = Namespace('http://dbpedia.org/property/')
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')
dbr = Namespace('http://dbpedia.org/resource/')
dbo = Namespace('http://dbpedia.org/ontology/')
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

# Bind the namespaces to prefixes in the graph
rdf_graph.bind("ex", ex)
rdf_graph.bind("dbo", dbo)
rdf_graph.bind("dbp", dbp)
rdf_graph.bind("rdf", RDF)
rdf_graph.bind("rdfs", RDFS)
sparql_endpoint = "http://dbpedia.org/sparql"


In [10]:
# Define the file path for the RDF graph try to use it to avoid time ofc using proper graph db would be efficient 
file_path = "1.ttl"

# Check if the file already exists
if os.path.exists(file_path):
    pass
else:
    # Load dataframe (df3_join) with the required columns: 'Artist Name', 'Album', 'Genre', 'Release Date', 'Genres', 'Average Rating'

    # Iterate over the dataframe rows and add triples to the graph
    for index, row in df3_join.iterrows():
        artist_name = row['Artist Name']
        album_name = row['Album']
        genre_name = row['Genre']
        release_date = row['Release Date']
        genres = row['Genres']
        average_rating = row['Average Rating']
        
        if not pd.isnull(artist_name):
            artist = URIRef(dbr[urllib.parse.quote(artist_name.replace(' ', '_'))])
        if not pd.isnull(album_name):
            album = URIRef(dbr[urllib.parse.quote(album_name.replace(' ', '_'))])
        if not pd.isnull(genre_name):
            genre = URIRef(ex[urllib.parse.quote(genre_name.replace(' ', '_'))])
        if not pd.isnull(genres):
            genres = URIRef(ex[urllib.parse.quote(genres.replace(' ', '_'))])

        try:
            if not pd.isnull(release_date):
                # Try to parse the release date using dateutil.parser
                parsed_date = parser.parse(release_date)
                xsd_date = rdflib.Literal(parsed_date.date(), datatype=rdflib.XSD.date)
        except (ValueError, OverflowError):
            # If parsing fails, add the release date as a string
            xsd_date = rdflib.Literal(release_date)

        rdf_graph.add((album, dbp.artist, artist))
        rdf_graph.add((album, rdfs.label, rdflib.Literal(album_name)))
        rdf_graph.add((album, dbp.releaseDate, xsd_date))
        rdf_graph.add((album, dbp.averageRating, rdflib.Literal(average_rating)))
        if not pd.isnull(genres):
            rdf_graph.add((genres, RDFS.subClassOf, genre))
        rdf_graph.add((album, RDF.type, genre))
        rdf_graph.add((album, RDF.type, dbo.MusicAlbum))



### **`2. Integrate your data with DBpedia's`**
                                                                                                                                                               
You MUST extend the knowledge you already have on your local KG with the following from DBpedia:     
- band members
- city of birth of members
- country of birth of members
- founding city of the band
- founding country of the band     

You can also add more information regarding albums (optional)



Recommendations and hints: 
- How to find IRIs from DBpedia that match the entities in the local KG?
  1) guessing the IRI with simple tricks:
     - the album 'Rubber Soul' on DBpedia is `dbr:Rubber_Soul`, and this 'translation' may work for other albums (don't do it manually) 
     - the album 'Led Zeppelin' on DBpedia is `dbr:Led_Zeppelin_(album)` as the same name is also associated to the band
  2) through their `rdfs:label`: for instance, the artist [Amy Winehouse](https://en.wikipedia.org/wiki/Amy_Winehouse) has 'Amy Winehouse' as label on DBpedia and therfore after the matching the IRI of the entity can be extracted
  3) using the [DBpedia lookup endpoint](https://github.com/dbpedia/lookup). The following request tries to find the DBpedia entity for the artist [Fabrizio De Andrè](https://en.wikipedia.org/wiki/Fabrizio_De_Andr%C3%A9): https://lookup.dbpedia.org/api/search/KeywordSearch?QueryString=fabrizio%20de%20andre&format=json&MaxHits=5
- restrict the search to entities of type 'Album', 'Band', 'country', etc, in order to avoid overloading DBPedia's endpoint (see [here](http://mappings.dbpedia.org/server/ontology/classes/) for a list of DBPedia classes) and thus avoid high latency or being banned      
- note that sometimes useful properties to reach entities have the `dbp` prefix which is different from `dbo` ([more info](https://parklize.blogspot.com/2016/05/dbpedia-difference-between-dbo-and-dbp.html)). It's ok to use both
- here is a **non-exaustive** list of useful properties: `dbp:hometown`, `dbo:birthPlace`, `dbo:country`, `dbo:formerBandMember`
- how to find properties that link two entities of our interest?
  1) by checking the [DBpedia ontology](https://akswnc7.informatik.uni-leipzig.de/dstreitmatter/archivo/dbpedia.org/ontology/2023.05.12-020000/ontology_type=parsed.owl) manually
  2) by checking the DBpedia ontology with Protégé
  3) using the DBpedia resource pages and exploiting your knowledge (or Wikipedia information). For instance, let us consider [Metallica](https://en.wikipedia.org/wiki/Metallica), I know that [Dave Mustaine](https://en.wikipedia.org/wiki/Dave_Mustaine) was a member, therefore I ask myself: How are these two entities linked in the DBpedia KG? On the [DBpedia page of Metallica](https://dbpedia.org/page/Metallica) we can find that the property used is `dbo:formerBandMember` thus I can use this property to get band members 
  4) running SPARQL queries like the following in the DBpedia endpoint:
      ```
      SELECT DISTINCT ?p
      WHERE {
        ?s a dbo:Band .
        ?o a dbo:MusicalArtist .
        ?s ?p ?o .
      }
      ```
  5) if you are curious enough you can check [ABSTAT](http://abstat.disco.unimib.it/about.html), a tool for KG exploration with a nice interface
- please notice that DBpedia is not perfect so is reccomendable to check the type of the candidates before including them

In [11]:
# can take up to 3 hours depends on the internet and the pc , recommendation deploy server would be better than querying dbpedia faster
# since will contain only small graph and more limited to the university so the connection rate is more 
if not os.path.exists(file_path):     
    #  file doesn't exist, perform SPARQL queries and add data to the graph

    # Initialize SPARQL wrapper
    sparql = SPARQLWrapper("https://dbpedia.org/sparql")

    # Define the query template
    query_template = '''
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dbo: <http://dbpedia.org/ontology/>

        ASK
        WHERE {{
            {{
                {{ <{artist}> a dbo:Band . }}
                UNION
                {{ <{artist}> dbo:wikiPageRedirects ?redirectedArtist . ?redirectedArtist a dbo:Band . }}
            }}
            UNION
            {{
                {{ <{artist}> a dbo:Person . FILTER NOT EXISTS {{ <{artist}> a dbo:Band }} }}
                UNION
                {{ <{artist}> dbo:wikiPageRedirects ?redirectedArtist . ?redirectedArtist a dbo:Person . FILTER NOT EXISTS {{ ?redirectedArtist a dbo:Band }} }}
            }}
        }}
    '''

    # Define a sample size
    sample_size = 100

    # Get a random sample from the dataframe
    df_sample = df3_join.sample(n=sample_size)  # was for fast debugging

    # Iterate over the sample rows and query DBpedia for additional information
    for index, row in df3_join.iterrows():
        artist_name = row['Artist Name']

        # Create the artist URI
        if not pd.isnull(artist_name):
            artist_uri = "http://dbpedia.org/resource/" + urllib.parse.quote(artist_name.replace(' ', '_'))

            # Prepare the SPARQL query
            query = query_template.format(artist=artist_uri)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)

            # use try-except to catch any error
            try:
                # Execute the query and process the result
                result = sparql.query().convert()
                is_band = result["boolean"]

                if is_band:
                    # The artist is a band, execute the band-related queries
                    band_query = '''
                        PREFIX dbp: <http://dbpedia.org/property/>
                        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                        PREFIX dbo: <http://dbpedia.org/ontology/>

                        SELECT DISTINCT ?member ?birthplace ?member_country ?founding_city ?founding_country
                        WHERE {{
                            {{
                                <{artist}> a dbo:Band ;
                                            dbo:formerBandMember ?member .
                            }}
                            UNION
                            {{
                                <{artist}> dbo:wikiPageRedirects ?redirectedArtist .
                                ?redirectedArtist a dbo:Band ;
                                                dbo:formerBandMember ?member .
                            }}
                            OPTIONAL {{ ?member dbo:birthPlace ?birthplace .}}
                            OPTIONAL {{ ?birthplace dbo:country ?member_country .}}
                            OPTIONAL {{ <{artist}> dbo:hometown ?founding_city.}}
                            OPTIONAL {{ ?redirectedArtist dbo:hometown ?founding_city.}}
                            OPTIONAL {{ ?founding_city dbo:country ?founding_country.}}
                        }} LIMIT 7
                    '''

                    # Prepare the band-related SPARQL query
                    band_query = band_query.format(artist=artist_uri)
                    sparql.setQuery(band_query)
                    sparql.setReturnFormat(JSON)

                    # Execute the query and process the results
                    results = sparql.query().convert()
                    for result in results["results"]["bindings"]:
                        member_uri = result.get("member", {}).get("value")
                        birthplace = result.get("birthplace", {}).get("value")
                        member_country = result.get("member_country", {}).get("value")
                        founding_city = result.get("founding_city", {}).get("value")
                        founding_country = result.get("founding_country", {}).get("value")

                        # Add the additional information to the graph
                        if member_uri:
                            rdf_graph.add((rdflib.URIRef(artist_uri), dbp.formerBandMember, rdflib.URIRef(member_uri)))

                        if birthplace:
                            rdf_graph.add((rdflib.URIRef(member_uri), dbp.birthPlace, rdflib.URIRef(birthplace)))

                        if member_country:
                            rdf_graph.add((rdflib.URIRef(member_uri), dbp.birthCountry, rdflib.URIRef(member_country)))

                        if founding_city:
                            rdf_graph.add((rdflib.URIRef(artist_uri), dbp.hometown, rdflib.URIRef(founding_city)))

                        if founding_country:
                            rdf_graph.add((rdflib.URIRef(artist_uri), dbp.founding_country, rdflib.URIRef(founding_country)))
                        
                else:
                    # The artist is not a band, execute the artist-related queries
                    artist_query = '''
                        PREFIX dbp: <http://dbpedia.org/property/>
                        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                        PREFIX dbo: <http://dbpedia.org/ontology/>

                        SELECT DISTINCT ?birthplace ?member_country
                        WHERE {{
                            {{
                                <{artist}> a dbo:Person ;
                                            dbo:birthPlace ?birthplace .
                            }}
                            UNION
                            {{
                                <{artist}> dbo:wikiPageRedirects ?redirectedArtist .
                                ?redirectedArtist a dbo:Person ;
                                                dbo:birthPlace ?birthplace .
                            }}
                            ?birthplace dbo:country ?member_country .
                            FILTER NOT EXISTS {{ <{artist}> a dbo:Band }}
                        }} LIMIT 7
                    '''

                    # Prepare the artist-related SPARQL query
                    artist_query = artist_query.format(artist=artist_uri)
                    sparql.setQuery(artist_query)
                    sparql.setReturnFormat(JSON)

                    # Execute the query and process the results
                    results = sparql.query().convert()
                    for result in results["results"]["bindings"]:
                        birthplace = result.get("birthplace", {}).get("value")
                        member_country = result.get("member_country", {}).get("value")

                        # Add the additional information to the graph
                        if birthplace:
                            rdf_graph.add((rdflib.URIRef(artist_uri), dbp.birthPlace, rdflib.URIRef(birthplace)))

                        if member_country:
                            rdf_graph.add((rdflib.URIRef(artist_uri), dbp.birthCountry, rdflib.URIRef(member_country)))

            except Exception as e:
                print(f"Error occurred for artist '{artist_name}': {str(e)}")


### **`3. Gathering Information from unstructured data`**

Knowledge can be gathered from unstructured data as well as structured data. A great encyclopaedic source is Wikipedia whose articles follow a very strict pattern. For instance, the first lines about a Person are reserved for biographical data and his or her occupation, therefore we can exploit it to understand the role of a band member and add this information to our KG.

In steps:
1) get the article from Wikipedia and extract the summary. You can use [Wikipedia-api](https://pypi.org/project/Wikipedia-API/) (see the code below)
2) extract the band member role (guitarist, bassist, keyboardist, singer, drummer, etc) using rules
3) add this information to the KG

Recommendations and hints: 
- some artists have multiple roles, add them all
- some artists have multiple bands, it is not required to understand the role for each band, keep it simple.
- extraction rules can be very simple (simple string matchers + if-else)  or more complex ones (e.g., regex, ChatGPT requests), it doesn't matter in terms of points.


In [12]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('en')
page_py = wiki_wiki.page('Kurt Cobain')

if page_py.exists():
    summary = page_py.summary[:200]
    print("Page - Summary: %s" % summary)
else:
    print("Page does not exist.")

Page - Summary: Kurt Donald Cobain (February 20, 1967 – c. April 5, 1994) was an American musician who was the co-founder, lead vocalist, guitarist and primary songwriter of the rock band Nirvana. Through his angst-f


In [13]:
# Initialize Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia('en')

# Check if the  file exists
if not os.path.exists(file_path):

    # Regular expression pattern for band member roles
    role_pattern = r"\b(guitarist|bassist|keyboardist|singer|drummer|DJ|artist|musician|rapper|producer|composer|vocalist|pianist|violinist|cellist|percussionist)\b"

    # SPARQL query to retrieve the former band members
    sparql_query = """
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX ex: <http://example.org/role/>
        SELECT ?artist ?member
        WHERE {
            ?artist dbp:formerBandMember ?member .
        }
    """

    # Execute the SPARQL query on the RDF graph
    results = rdf_graph.query(sparql_query)

    # Loop through the SPARQL query results
    for row in results:
        artist_uri, member_uri = row

        # Extract the artist name and member name from the URIs
        artist_name = str(artist_uri).split("/")[-1]
        member_name = str(member_uri).split("/")[-1]

        retries = 3  # Number of retries if a timeout occurs
        for retry in range(retries):
            try:
                # Retrieve the Wikipedia page
                page_py = wiki_wiki.page(member_name)

                # Check if the page exists
                if page_py.exists():
                    # Get the summary of the page
                    summary = page_py.summary

                    # Find all band member roles in the summary
                    roles = re.findall(role_pattern, summary)

                    # Update the graph with the roles for the current member
                    member_uri = URIRef(member_uri)
                    for role in roles:
                        role_uri = URIRef("http://example.org/role/{}".format(role))
                        rdf_graph.add((member_uri, URIRef("http://dbpedia.org/ontology/hasrole"), role_uri))

                    break  # Break the retry loop if successful

                else:
                    print("Page for {} does not exist.".format(member_name))

            except ReadTimeout:
                if retry == retries - 1:
                    print("Timeout occurred while retrieving page for {}. Retries exhausted.".format(member_name))
                else:
                    print("Timeout occurred while retrieving page for {}. Retrying...".format(member_name))
                    time.sleep(1)  # Wait for a while before retrying


### **`4. Include a small ontology and a  genres taxonomy`**

Using RDFlib, add classes such as Person, MusicAlbum, and so on to your KG. Add the domain and range for each property you use.

This will be needed later for inferring the types of the entities of your KG.

In [14]:
# Your solution ...
rdf_graph.add((ex.role, RDF.type, RDFS.Class))
rdf_graph.add((ex.country, RDF.type, RDFS.Class))
rdf_graph.add((dbo.Person, RDF.type, RDFS.Class))
rdf_graph.add((dbo.Band, RDF.type, RDFS.Class))
rdf_graph.add((dbo.MusicAlbum, RDF.type, RDFS.Class))
rdf_graph.add((dbp.formerBandMember, RDF.type, RDF.Property))
rdf_graph.add((dbp.founding_country , RDF.type, RDF.Property))
rdf_graph.add((dbp.hometown, RDF.type, RDF.Property))
rdf_graph.add((dbp.birthPlace, RDF.type, RDF.Property))
rdf_graph.add((dbp.birthCountry, RDF.type, RDF.Property))
rdf_graph.add((dbo.hasrole, RDF.type, RDF.Property))

<Graph identifier=Nfe362c4c4fdd4aa580a37be148785952 (<class 'rdflib.graph.Graph'>)>

Moreover, the first dataframe has 'Genre' and 'Subgenre' columns.
Extract a genres taxonomy from the dataset and include it in your RDF KG. This will be needed later for inferring genres from subgenres.

E.g.: add triples to the graph like:

- `[subgenre] rdfs:subClassOf [genre]`.
- `[album] rdf:type [genre]`.

This should be done for all genres and subgenres (with an iteration, not manually).

In [15]:
# Your solution ...
#subClassof was added in the for loop in the upper cells
rdf_graph.add((dbo.hasrole, RDFS.domain, dbo.Person))     
rdf_graph.add((dbo.hasrole, RDFS.range, ex.role))
rdf_graph.add((dbp.birthCountry, RDFS.domain, dbo.Person))
rdf_graph.add((dbp.birthCountry, RDFS.range, ex.country))
rdf_graph.add((dbp.founding_country, RDFS.domain, dbo.Band))
rdf_graph.add((dbp.founding_country, RDFS.range, ex.country))


<Graph identifier=Nfe362c4c4fdd4aa580a37be148785952 (<class 'rdflib.graph.Graph'>)>

### **`5. Materialize inferences`**

Add to your KG inferences that you can make from the data on your KG (e.g., with SPARQL queries, or with OWLRL, ...).

**<ins>Some</ins>**  **<ins>examples</ins>**
- if you have that `[album] rdf:type [genre1]` and `[genre1] rdfs:subClassOf [genre2]`, add the triple `[album] rdf:type [genre2]`. Same for domains, ranges, subclasses, subproperties, OWL features, etc.
- if the nationality of a group member is missing, you can infer it from the nationality of the band

Run a few SPARQL queries on the graph to see the inferred triples.


In [16]:
if not os.path.exists(file_path):
    # Define the SPARQL query for constructing additional triples based on subclass relationships
    query = '''
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      CONSTRUCT {
          ?album rdf:type ?genre2 .
      }
      WHERE {
          ?album rdf:type ?genre1 .
          ?genre2 rdfs:subClassOf ?genre1 .
      }
    '''

    # Prepare the SPARQL query
    prepared_query = prepareQuery(query, initNs={"rdf": RDF, "rdfs": RDFS})

    # Execute the SPARQL query on the RDF graph
    constructed_graph = rdf_graph.query(prepared_query).graph

    # Add the constructed triples to the original RDF graph
    rdf_graph += constructed_graph

    # Define the SPARQL query for inserting domain and range triples
    sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    INSERT {
      ?something rdf:type ?type .
    }
    WHERE {
      {
        ?relation rdfs:domain ?type .
        ?something ?relation ?withotherthing .
      }
      UNION
      {
        ?relation rdfs:range ?type .
        ?withotherthing ?relation ?something .
      }
    }
    """

    # Execute the SPARQL query and insert the types directly into the graph
    rdf_graph.update(sparql_query)


In [17]:
if not os.path.exists(file_path):
  sparql_query = """
  PREFIX dbo: <http://dbpedia.org/ontology/>
  PREFIX dbp: <http://dbpedia.org/property/>

  SELECT  ?band ?foundingcountry ?member ?birthCountry
  WHERE {
    ?band rdf:type dbo:Band .
    ?band dbp:formerBandMember ?member .
    ?band dbp:founding_country ?foundingcountry .
    OPTIONAL {
      ?member dbp:birthCountry ?birthCountry .
    }
    FILTER NOT EXISTS { ?member dbp:birthCountry [] }
  }
  """

  # Execute the SPARQL query and retrieve the results
  results = rdf_graph.query(sparql_query)

  # Create a list to store the members without a country
  members_without_country = []

  # Iterate over the results
  for result in results:
      band = result["band"]
      founding_country = result["foundingcountry"]
      member = result["member"]
      birth_country = result["birthCountry"]
      

      # Check if the former member's country is missing
      if member is not None:
          # Store the member in the list
          members_without_country.append(member)

  # Assign the founding country of the band to each member without a country
  for member in members_without_country:
      rdf_graph.add((member, dbp.birthCountry, founding_country))
  
  # Save the RDF graph to the TT1 file
  rdf_graph.serialize(destination=file_path, format='turtle')
else:
    rdf_graph.parse(file_path, format='turtle')


### **`6. Query the Graph`**

Query your graph in a way that:

* the usefulness of having added information from dbpedia is highlighted
* inference is exploited

<b><u>Some</u></b> <b><u>examples</u></b> with the intentions:

| ID | Query | Intention/insight |
| --- | ----- | ----------------- |
| 1 | which bands have at least one member with a different nationality? | just curiosity |
| 2 | Which are the artist with many bands? | to spot session musicians, long-career artists, 'eclectic' artists |
| 3 | which are the countries with more [Latin Rock](https://en.wikipedia.org/wiki/Latin_rock) bands? and what about [Black Metal](https://en.wikipedia.org/wiki/Black_metal) ? | I suppose that latin rock is more popular in Latin America countries while Black Metal in Scandinavian countries, but I may be wrong! |
| 4 | Is there any artist who is a member of both a [Britpop](https://en.wikipedia.org/wiki/Britpop) band and a [Hip hop](https://en.wikipedia.org/wiki/Hip_hop_music) band | maybe to spot bands/artists with a strong musical contamination? e.g., [Damon Albarn](https://en.wikipedia.org/wiki/Damon_Albarn) |
| 5 | Which is the city with more bands in absolute? and what if we consider only [Grunge](https://en.wikipedia.org/wiki/Grunge) ? | maybe to discover a music scene or just a very influential city. |
| 6 | Give me some band/artist from the same city of [2Pac](https://en.wikipedia.org/wiki/Tupac_Shakur) | could be a way to find similar musical artists. What about adding also a time period and the musical genre? |
| 7 | which are the bands with Argentine and Spanish members? | maybe because I love both the Argentine and Spanish sound and I hope there is something that matches both? e.g., [Los Rodriguez](https://en.wikipedia.org/wiki/Los_Rodr%C3%ADguez) |

**Feel free to play and get cool insights**

In [18]:
# Your solution ...
# 1
# Define the SPARQL query to retrieve bands with members of different nationalities
sparql_query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT DISTINCT ?band
WHERE {
  ?band rdf:type dbo:Band .
  ?band dbp:founding_country ?foundingcountry .
  ?band dbp:formerBandMember ?member .
  ?member dbp:birthCountry ?birthCountry .
  FILTER (?birthCountry != ?foundingcountry)
}
"""

# Execute the SPARQL query and retrieve the results
results = rdf_graph.query(sparql_query)

# Iterate over the results and print the bands
for result in results:
    band = result["band"]
    print(band)


http://dbpedia.org/resource/Afro-Cuban_All_Stars
http://dbpedia.org/resource/Agitation_Free
http://dbpedia.org/resource/Angel_Witch
http://dbpedia.org/resource/Armored_Saint
http://dbpedia.org/resource/Art_Blakey_and_the_Jazz_Messengers
http://dbpedia.org/resource/Art_Ensemble_of_Chicago
http://dbpedia.org/resource/Ben_Folds_Five
http://dbpedia.org/resource/Big_Big_Train
http://dbpedia.org/resource/Black_Uhuru
http://dbpedia.org/resource/Blind_Faith
http://dbpedia.org/resource/British_Sea_Power
http://dbpedia.org/resource/Buena_Vista_Social_Club
http://dbpedia.org/resource/Camberwell_Now
http://dbpedia.org/resource/Choking_Victim
http://dbpedia.org/resource/Christian_Death
http://dbpedia.org/resource/Crass
http://dbpedia.org/resource/Deafheaven
http://dbpedia.org/resource/Derek_and_The_Dominos
http://dbpedia.org/resource/ELO
http://dbpedia.org/resource/Eddie_and_the_Hot_Rods
http://dbpedia.org/resource/Evergrey
http://dbpedia.org/resource/Everything_Everything
http://dbpedia.org/resour

In [28]:
#2   # I can add other relation to make artist subclass of band
# Define the SPARQL query to retrieve artists with multiple bands/ own album
sparql_query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT ?artist (COUNT(?band) AS ?numBands) (COUNT(?album) AS ?numsinglealbum)
WHERE {
  ?album dbp:artist ?artist.
  ?artist rdf:type dbo:Person .
  ?band dbp:formerBandMember ?artist .
}
GROUP BY ?artist
HAVING (COUNT(?band) > 1)
ORDER BY DESC(COUNT(?band))
"""

# Execute the SPARQL query and retrieve the results
results = rdf_graph.query(sparql_query)

# Iterate over the results and print the artists with their number of bands
for result in results:
    artist = result["artist"]
    num_bands = result["numBands"]
    num_single_album = result['numsinglealbum']
    print(f"Artist: {artist}, Number of Bands: {num_bands}, Number of own made album {num_single_album}")



Artist: http://dbpedia.org/resource/The_Mothers_of_Invention, Number of Bands: 12, Number of own made album 12
Artist: http://dbpedia.org/resource/Bruce_Springsteen, Number of Bands: 7, Number of own made album 7
Artist: http://dbpedia.org/resource/Mark_Lanegan, Number of Bands: 6, Number of own made album 6
Artist: http://dbpedia.org/resource/Sandy_Denny, Number of Bands: 6, Number of own made album 6
Artist: http://dbpedia.org/resource/Curtis_Mayfield, Number of Bands: 5, Number of own made album 5
Artist: http://dbpedia.org/resource/Gram_Parsons, Number of Bands: 5, Number of own made album 5
Artist: http://dbpedia.org/resource/Brian_Eno, Number of Bands: 3, Number of own made album 3
Artist: http://dbpedia.org/resource/Dr._Dre, Number of Bands: 3, Number of own made album 3
Artist: http://dbpedia.org/resource/Ice_Cube, Number of Bands: 3, Number of own made album 3
Artist: http://dbpedia.org/resource/Kamelot, Number of Bands: 3, Number of own made album 3
Artist: http://dbpedia.org

In [20]:
#3 
# Define the SPARQL query to retrieve countries with more Latin Rock bands
sparql_query_latin_rock = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX ex: <http://example.org/>

SELECT ?country (COUNT(?band) AS ?numBands)
WHERE {
  ?band rdf:type dbo:Band .
  ?band dbp:founding_country ?country .
  ?band rdf:type ex:Latin_Rock .
}
GROUP BY ?country
ORDER BY DESC(COUNT(?band))
"""

# Execute the SPARQL query for Latin Rock bands and retrieve the results
results_latin_rock = rdf_graph.query(sparql_query_latin_rock)

# Iterate over the results and print the countries with their number of Latin Rock bands
for result in results_latin_rock:
    country = result["country"]
    num_bands = result["numBands"]
    print(f"country: {country}, Number of Latin Rock Bands: {num_bands}")


Country: http://dbpedia.org/resource/United_Kingdom, Number of Latin Rock Bands: 23
Country: http://dbpedia.org/resource/United_States, Number of Latin Rock Bands: 20
Country: http://dbpedia.org/resource/Australia, Number of Latin Rock Bands: 7
Country: http://dbpedia.org/resource/Argentina, Number of Latin Rock Bands: 5
Country: http://dbpedia.org/resource/India, Number of Latin Rock Bands: 5
Country: http://dbpedia.org/resource/Kent, Number of Latin Rock Bands: 3
Country: http://dbpedia.org/resource/Germany, Number of Latin Rock Bands: 2
Country: http://dbpedia.org/resource/Brazil, Number of Latin Rock Bands: 2
Country: http://dbpedia.org/resource/Surrey, Number of Latin Rock Bands: 1
Country: http://dbpedia.org/resource/Italy, Number of Latin Rock Bands: 1
Country: http://dbpedia.org/resource/Chile, Number of Latin Rock Bands: 1


In [21]:
#4
# Define the SPARQL query to retrieve countries with more Black Metal bands
sparql_query_black_metal = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX ex: <http://example.org/>

SELECT ?country (COUNT(?band) AS ?numBands)
WHERE {
  ?band rdf:type dbo:Band .
  ?band dbp:founding_country ?country .
  ?band rdf:type ex:Black_metal .
}
GROUP BY ?country
ORDER BY DESC(COUNT(?band))
"""

# Execute the SPARQL query for Black Metal bands and retrieve the results
results_black_metal = rdf_graph.query(sparql_query_black_metal)

# Iterate over the results and print the countries with their number of Black Metal bands
for result in results_black_metal:
    country = result["country"]
    num_bands = result["numBands"]
    print(f"country: {country}, Number of Black Metal Bands: {num_bands}")

# Define the SPARQL query to retrieve artists who are members of both Britpop and Hip hop bands
sparql_query = """
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX ex: <http://example.org/>

SELECT DISTINCT ?artist
WHERE {
  ?britpopBand dbp:formerBandMember ?artist .
  ?hiphopBand dbp:formerBandMember ?artist .
  ?britpopBand rdf:type ex:Britpop.
  ?hiphopBand rdf:type ex:Hip_Hop .
}
"""

# Execute the SPARQL query and retrieve the results
results = rdf_graph.query(sparql_query)

# Iterate over the results and print the artists
for result in results:
    artist = result["artist"]
    print(f"Band: {artist}")


Band: http://dbpedia.org/resource/Lee_Altus
Band: http://dbpedia.org/resource/Manuel_Göttsching
Band: http://dbpedia.org/resource/Eric_Clapton
Band: http://dbpedia.org/resource/Ginger_Baker
Band: http://dbpedia.org/resource/Ric_Grech
Band: http://dbpedia.org/resource/Steve_Winwood
Band: http://dbpedia.org/resource/Amadito_Valdés
Band: http://dbpedia.org/resource/Lee_Dorman
Band: http://dbpedia.org/resource/Sandy_Denny
Band: http://dbpedia.org/resource/Trevor_Lucas
Band: http://dbpedia.org/resource/Craig_Frost
Band: http://dbpedia.org/resource/Dennis_Bellinger
Band: http://dbpedia.org/resource/Mark_Farner
Band: http://dbpedia.org/resource/Rainer_Loskand
Band: http://dbpedia.org/resource/Bill_MacCormick
Band: http://dbpedia.org/resource/Dave_MacRae
Band: http://dbpedia.org/resource/Phil_Miller
Band: http://dbpedia.org/resource/Tony_Newman_(drummer)
Band: http://dbpedia.org/resource/Ani_Cordero
Band: http://dbpedia.org/resource/Arnaldo_Baptista
Band: http://dbpedia.org/resource/Zélia_Dunc

In [22]:
#5
# Define the SPARQL query to retrieve cities and the count of associated bands
sparql_query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT ?city (COUNT(?band) AS ?bandCount)
WHERE {
  ?band rdf:type dbo:Band .
  ?band dbp:hometown ?city .
}
GROUP BY ?city
ORDER BY DESC(?bandCount)
LIMIT 1
"""

# Execute the SPARQL query and retrieve the result
result = rdf_graph.query(sparql_query).bindings[0]
city = result["city"]
band_count = result["bandCount"]

print(f"City with the most bands in absolute: {city}, Band count: {band_count}")


City with the most bands in absolute: http://dbpedia.org/resource/London_Borough_of_Camden, Band count: 42


In [30]:
#6
# Define the SPARQL query to retrieve the birthplace of 2Pac
sparql_query_birthplace = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT ?city
WHERE {
  <http://dbpedia.org/resource/Tupac_Shakur> dbo:birthPlace ?city .
}
"""

# Create a SPARQLWrapper instance and set the endpoint URL
sparql = SPARQLWrapper(sparql_endpoint)

# Set the SPARQL query string
sparql.setQuery(sparql_query_birthplace)

# Set the return format to JSON
sparql.setReturnFormat('json')

# Execute the SPARQL query and retrieve the results
query_results = sparql.query().convert()

# Extract the bindings from the query results
bindings = query_results["results"]["bindings"]

# Extract the birthplace city of 2Pac
birthplace_city = bindings[0]["city"]["value"] if bindings else None
if birthplace_city:
    # Define the SPARQL query to retrieve bands/artists from the same city as 2Pac
    sparql_query_artists = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbp: <http://dbpedia.org/property/>
    
    SELECT ?artist 
    WHERE {{
    {{
      ?artist rdf:type dbo:Band .
      ?artist dbp:hometown <{birthplace_city}>.
    }}
    UNION 
    {{
      ?artist rdf:type dbo:Person.
      ?artist dbp:birthPlace <{birthplace_city}>.
    }}

    }}
    """

    # Query the SPARQL endpoint and retrieve the results
    results = rdf_graph.query(sparql_query_artists)
    

    # Iterate over the results and print the bands/artists from the same city as 2Pac
    for result in results:
        artist = result["artist"]
        print(f"Artist/Band: {artist}, with same birthplace/hometown: {birthplace_city}")
else:
    print("Unable to determine the birthplace city of 2Pac.")


Artist/Band: http://dbpedia.org/resource/Sharon_Jones_and_The_Dap-Kings, with same birthplace/hometown: http://dbpedia.org/resource/New_York_City
Artist/Band: http://dbpedia.org/resource/The_Ronettes, with same birthplace/hometown: http://dbpedia.org/resource/New_York_City
Artist/Band: http://dbpedia.org/resource/Vampire_Weekend, with same birthplace/hometown: http://dbpedia.org/resource/New_York_City
Artist/Band: http://dbpedia.org/resource/Wu-Tang_Clan, with same birthplace/hometown: http://dbpedia.org/resource/New_York_City
Artist/Band: http://dbpedia.org/resource/Mahavishnu_Orchestra, with same birthplace/hometown: http://dbpedia.org/resource/New_York_City
Artist/Band: http://dbpedia.org/resource/Mobb_Deep, with same birthplace/hometown: http://dbpedia.org/resource/New_York_City
Artist/Band: http://dbpedia.org/resource/A_Tribe_Called_Quest, with same birthplace/hometown: http://dbpedia.org/resource/New_York_City
Artist/Band: http://dbpedia.org/resource/Simon_and_Garfunkel, with sam

In [29]:
#7 no answer doesn't have this kind of data
sparql_query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT DISTINCT ?band
WHERE {
  ?band rdf:type dbo:Band .
  ?band dbp:formerBandMember ?member1 .
  ?band dbp:formerBandMember ?member2 .
  ?member1 dbp:birthCountry dbr:Argentina .
  ?member2 dbp:birthCountry dbr:Spain .
  FILTER(?member1 != ?member2)
}
"""

# Create a SPARQLWrapper instance and set the endpoint URL
sparql = SPARQLWrapper(sparql_endpoint)

# Set the SPARQL query string
sparql.setQuery(sparql_query)

# Set the return format to JSON
sparql.setReturnFormat('json')

# Execute the SPARQL query and retrieve the results
query_results = sparql.query().convert()

# Extract the bindings from the query results
bindings = query_results["results"]["bindings"]

# Iterate over the bindings and print the bands with Argentine and Spanish members
for binding in bindings:
    band = binding["band"]["value"]
    print(f"Band: {band}")